#Boston 311 Tutorial

This notebook will run you through the basic usage of this package to train 3 models on the Boston 311 data and use them to predict the outcome of cases from the last 30 days

In [17]:
! pip install ../

Defaulting to user installation because normal site-packages is not writeable
Processing /home/briarmoss/Documents/Boston_311
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for boston311: filename=boston311-0.1.0-py3-none-any.whl size=16081 sha256=c4cea62fb5aad2b62def42bee1d54b6f25068a88bb2c98df406a26eef41d7204
  Stored in directory: /tmp/pip-ephem-wheel-cache-uugfpdrq/wheels/3d/69/ee/0a6ac96b9c09c948fc0e74f2724a9703aa39749a41fa757c9e
Successfully built boston311
  Attempting uninstall: boston311
    Found existing installation: boston311 0.1.0
    Uninstalling boston311-0.1.0:
      Successfully uninstalled boston311-0.1.0


##Import the Boston311Model class

In [18]:
import os

In [19]:
from boston311 import Boston311LogReg, Boston311EventDecTree, Boston311SurvDecTree


## Get latest file URLS and Current Date Ranges

In [20]:
latest_URLS = Boston311LogReg.get311URLs()

In [21]:
print(latest_URLS)

{'2023': 'https://data.boston.gov/dataset/8048697b-ad64-4bfc-b090-ee00169f2323/resource/e6013a93-1321-4f2a-bf91-8d8a02f1e62f/download/tmp4i_c_2hr.csv', '2022': 'https://data.boston.gov/dataset/8048697b-ad64-4bfc-b090-ee00169f2323/resource/81a7b022-f8fc-4da5-80e4-b160058ca207/download/tmpfm8veglw.csv', '2021': 'https://data.boston.gov/dataset/8048697b-ad64-4bfc-b090-ee00169f2323/resource/f53ebccd-bc61-49f9-83db-625f209c95f5/download/tmp88p9g82n.csv', '2020': 'https://data.boston.gov/dataset/8048697b-ad64-4bfc-b090-ee00169f2323/resource/6ff6a6fd-3141-4440-a880-6f60a37fe789/download/tmpcv_10m2s.csv', '2019': 'https://data.boston.gov/dataset/8048697b-ad64-4bfc-b090-ee00169f2323/resource/ea2e4696-4a2d-429c-9807-d02eb92e0222/download/tmpcje3ep_w.csv', '2018': 'https://data.boston.gov/dataset/8048697b-ad64-4bfc-b090-ee00169f2323/resource/2be28d90-3a90-4af1-a3f6-f28c1e25880a/download/tmp7602cia8.csv', '2017': 'https://data.boston.gov/dataset/8048697b-ad64-4bfc-b090-ee00169f2323/resource/300221

In [22]:
from datetime import datetime, timedelta
now = datetime.now()
thirty_days = timedelta(days=30)
thirty_days_ago = now - thirty_days
today_datestring = now.strftime("%Y-%m-%d")
thirty_days_ago_datestring = thirty_days_ago.strftime("%Y-%m-%d")
tomorrow_datestring = (datetime.today() + timedelta(days=1)).strftime('%Y-%m-%d')

print(today_datestring, thirty_days_ago_datestring, tomorrow_datestring)

2023-09-19 2023-08-20 2023-09-20


In [23]:
#set model folder constant
MODEL_FOLDER = './daily_models'

##Define several models

In [33]:
from typing import List, Dict, Any
from itertools import combinations

# Function to define an iteration scenario
def define_iteration_scenario(feature_columns: List[str], time_spans: List[Dict[str, str]]) -> Dict[str, Any]:
    scenario = {
        'feature_columns': feature_columns,
        'time_spans': time_spans
    }
    return scenario

# Function to produce all the models based on an iteration scenario
def generate_models(scenario: Dict[str, Any], latest_urls: Dict[str, str], model_class, combination_size=1):
    generated_models = {}

    scenario_dict = {'dropColumnValues': {'source': ['City Worker App', 'Employee Generated']}, 'survivalTimeMin': 0}
    if model_class == Boston311SurvDecTree:
        scenario_dict['survivalTimeFill'] = tomorrow_datestring
    
    # Iterate through all paired combinations of feature columns
    for feature_pair in combinations(scenario['feature_columns'], combination_size):
        
        # Iterate through all time spans
        for time_span in scenario['time_spans']:
            train_start = time_span['start']
            train_end = time_span['end']
            
            # Create a model
            model = model_class(
                train_date_range={'start': train_start, 'end': train_end},
                predict_date_range={'start': thirty_days_ago_datestring, 'end': tomorrow_datestring},  # Adjust as needed
                feature_columns=list(feature_pair),
                scenario=scenario_dict, 
                files_dict=latest_urls
            )
            
            #create variable to hold model name and add the model_class.__name__
            model_name = model_class.__name__
            #append the feature pair by iterating over it
            for feature in feature_pair:
                model_name += f'_{feature}'

            #store model with key as class name and feature pair
            generated_models[model_name] = model
            
    return generated_models

# Example usage
feature_columns_to_use = ['type', 'queue', 'source', 'subject', 'reason', 'department', 'ward_number']  # Replace with your actual feature columns

#feature_columns_to_use = ['queue', 'ward_number'] 

time_spans_to_use = [
    {'start': '2022-01-01', 'end': thirty_days_ago_datestring},
]  # Replace with your actual time spans

# Define the iteration scenario
iteration_scenario = define_iteration_scenario(feature_columns_to_use, time_spans_to_use)

# Generate the models (replace `Boston311LogReg` with the model class you want to use)
# latest_URLS would be the actual URLs you have
#generated_models = generate_models(iteration_scenario, latest_urls=latest_URLS, model_class=Boston311LogReg, combination_size=1)
#Add the other models
#generated_models.update(generate_models(iteration_scenario, latest_urls=latest_URLS, model_class=Boston311EventDecTree, combination_size=1))
#generated_models.update(generate_models(iteration_scenario, latest_urls=latest_URLS, model_class=Boston311SurvDecTree, combination_size=1))
#run the same but with combination_size = 2
#generated_models.update(generate_models(iteration_scenario, latest_urls=latest_URLS, model_class=Boston311LogReg, combination_size=2))
#generated_models.update(generate_models(iteration_scenario, latest_urls=latest_URLS, model_class=Boston311EventDecTree, combination_size=2))
#generated_models.update(generate_models(iteration_scenario, latest_urls=latest_URLS, model_class=Boston311SurvDecTree, combination_size=2))
# Now, `generated_models` contains all the models based on the defined iteration scenario
generated_models = (generate_models(iteration_scenario, latest_urls=latest_URLS, model_class=Boston311SurvDecTree, combination_size=6))
#print model keys
#print(generated_models.keys())

#for each model in generated_models print the scenario
for model in generated_models:
    print(generated_models[model].scenario)
    #print newline
    print('\n')

len(generated_models) 


{'dropColumnValues': {'source': ['City Worker App', 'Employee Generated']}, 'survivalTimeMin': 0, 'survivalTimeFill': '2023-09-20'}


{'dropColumnValues': {'source': ['City Worker App', 'Employee Generated']}, 'survivalTimeMin': 0, 'survivalTimeFill': '2023-09-20'}


{'dropColumnValues': {'source': ['City Worker App', 'Employee Generated']}, 'survivalTimeMin': 0, 'survivalTimeFill': '2023-09-20'}


{'dropColumnValues': {'source': ['City Worker App', 'Employee Generated']}, 'survivalTimeMin': 0, 'survivalTimeFill': '2023-09-20'}


{'dropColumnValues': {'source': ['City Worker App', 'Employee Generated']}, 'survivalTimeMin': 0, 'survivalTimeFill': '2023-09-20'}


{'dropColumnValues': {'source': ['City Worker App', 'Employee Generated']}, 'survivalTimeMin': 0, 'survivalTimeFill': '2023-09-20'}


{'dropColumnValues': {'source': ['City Worker App', 'Employee Generated']}, 'survivalTimeMin': 0, 'survivalTimeFill': '2023-09-20'}




7

In [34]:
#get current datetime in Boston timezone as string
from datetime import datetime
from pytz import timezone
import pytz
boston = timezone('US/Eastern')
now = datetime.now(boston)
today_datestring = now.strftime("%Y-%m-%d")
#get time in Boston timezone as string for a filename
now = datetime.now(boston)
time_string = now.strftime("%H-%M-%S")
#define datetime string
my_datetime = today_datestring + '_' + time_string 

## Train all the models

In [35]:
#define accuracy dict
model_accuracy = {}
#get the first model and save the results of the load_data method to a dataframe
model = list(generated_models.values())[0]
df = model.load_data()

for name, model in generated_models.items():
    #print the model we are training
    print("training model: " + name)
    test_acc = model.run_pipeline(df)
    #add model name and accuracy to dict
    model_accuracy[name] = test_acc
    #print Done training model
    print("Done training model: " + name + " accuracy: " + str(test_acc))
    

/home/briarmoss/.local/lib/python3.10/site-packages/boston311/Boston311Model.py:255: DtypeWarning: Columns (13) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file)
/home/briarmoss/.local/lib/python3.10/site-packages/boston311/Boston311Model.py:255: DtypeWarning: Columns (13) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file)


Files with different number of columns from File 0:  []
Files with same number of columns as File 0:  [0, 1]
Files with different column order from File 0:  []
Files with same column order as File 0:  [0, 1]
training model: Boston311SurvDecTree_type_queue_source_subject_reason_department
Starting Training at 2023-09-19 23:07:32.143879
Testing accuracy: 0.6945399828030955
Ending Training at 2023-09-19 23:07:38.592320
Training took 0:00:06.448441
Done training model: Boston311SurvDecTree_type_queue_source_subject_reason_department accuracy: 0.6945399828030955
training model: Boston311SurvDecTree_type_queue_source_subject_reason_ward_number
Starting Training at 2023-09-19 23:07:41.775942
Testing accuracy: 0.7000214961306965
Ending Training at 2023-09-19 23:07:50.498504
Training took 0:00:08.722562
Done training model: Boston311SurvDecTree_type_queue_source_subject_reason_ward_number accuracy: 0.7000214961306965
training model: Boston311SurvDecTree_type_queue_source_subject_department_ward

In [36]:
#sort the model accuracy dict by value
model_accuracy = dict(sorted(model_accuracy.items(), key=lambda item: item[1], reverse=True))

#print header for data
print("model_name, accuracy")

#print model accuracy dict nicely
for key, value in model_accuracy.items():
    print(key, ' : ', str(value))

model_name, accuracy
Boston311SurvDecTree_type_queue_subject_reason_department_ward_number  :  0.7008932836533869
Boston311SurvDecTree_type_queue_source_subject_reason_ward_number  :  0.7000214961306965
Boston311SurvDecTree_type_queue_source_subject_department_ward_number  :  0.7000214961306965
Boston311SurvDecTree_type_queue_source_reason_department_ward_number  :  0.6998304194133945
Boston311SurvDecTree_queue_source_subject_reason_department_ward_number  :  0.6965940575140919
Boston311SurvDecTree_type_queue_source_subject_reason_department  :  0.6945399828030955
Boston311SurvDecTree_type_source_subject_reason_department_ward_number  :  0.6822991306009363


In [37]:
#save the model accuracy dict to a csv
import csv
with open(f'{MODEL_FOLDER}/model_accuracy_{my_datetime}.csv', 'w') as f:
    for key in model_accuracy.keys():
        f.write("%s,%s\n"%(key,model_accuracy[key]))

In [38]:
import gc
gc.collect()

28

In [39]:
#run the predict method on each model

#for name, model in generated_models.items():
   #print the model we are predicting
#    print("predicting model: " + name)
#    model.predict()
    #print Dont predicting model
#    print("Done predicting model: " + name)

In [40]:

import datetime

def save_model_to_dir(model, folder_name):
    dir_path = os.path.join(MODEL_FOLDER, folder_name)
    
    if not os.path.exists(dir_path):
        os.mkdir(dir_path)
    
    timestamp = datetime.datetime.now().strftime('%Y%m%d_%H%M%S')
    model_name = timestamp + "_" + model.model_type
    properties_name = model_name
    
    model.save(dir_path, model_name, properties_name)


# Iterate over models and save
for model_name, model in generated_models.items():
    save_model_to_dir(model, model_name)
